# FlyingSquid

In [ ]:
#pip install sentence_transformers

In [1]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
train = dataset['train']
dev = dataset['validation']

records = [
    rb.TextClassificationRecord(
        inputs=text
    )
    for text in train['sentence']
]

records += [
    rb.TextClassificationRecord(
        inputs=text,
        annotation=dev.features['label'].names[label]
    )
    for text, label in zip(dev['sentence'], dev['label'])
]


rb.log(records, name="weak_supervision_sst2")

In [2]:
from rubrix.labeling.text_classification import Rule, WeakLabels
import rubrix as rb

positive_keywords = [
    "funny", "comedy", "love",
    "fun", "entertaining", "romantic",
    "compelling", "worth", "sweet",
    "fascinating", "laughs", "comic",
    "enjoyable", "clever", "perfect",
    "beautiful", "amusing", "powerful",
    "charming", "engaging", 
]

negative_keywords = [
    "bad", "dull", "worst", "worse",
    "spiritless", "silly", "monotonous", 
    "terrible", "banal", "unimaginative", 
    "inane", "shallow", "offensive", 
    "redundant", "lazy", "loose", 
    "poorly", "awful", "pathetic", 
    "lousy", "inept"
]

rules = [ Rule(query=keyword, label="positive") for keyword in positive_keywords ]
rules += [ Rule(query=keyword, label="negative") for keyword in negative_keywords ]

from rubrix.labeling.text_classification import load_rules

# optionally add the rules defined in the web app UI
rules += load_rules(dataset="weak_supervision_sst2")

# apply the rules to a dataset to obtain the weak labels
weak_labels = WeakLabels(
    rules=rules,
    dataset="weak_supervision_sst2"
)

from rubrix.labeling.text_classification import FlyingSquid

# we pass our WeakLabels instance to our FlyingSquid label model
flyingsquid_model = FlyingSquid(weak_labels)

# we fit the model
flyingsquid_model.fit()

flyingsquid_model.score(tie_break_policy="abstain")

/home/user/.local/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


Preparing rules:   0%|          | 0/41 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/68221 [00:00<?, ?it/s]

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


{'efficacy': 0.4733926941735914,
 'fscore_cautious': 0.33672608742524845,
 'coverage': 0.2190366972477064,
 'negative': {'precision': 0.9038461538461539,
  'recall': 0.5,
  'f1-score': 0.6438356164383562,
  'support': 94},
 'positive': {'precision': 0.6618705035971223,
  'recall': 0.9484536082474226,
  'f1-score': 0.7796610169491525,
  'support': 97},
 'accuracy': 0.7277486910994765,
 'macro avg': {'precision': 0.7828583287216381,
  'recall': 0.7242268041237113,
  'f1-score': 0.7117483166937544,
  'support': 191},
 'weighted avg': {'precision': 0.7809579963898394,
  'recall': 0.7277486910994765,
  'f1-score': 0.7128150083208026,
  'support': 191}}

In [33]:
import pandas as pd
import numpy as np
report = flyingsquid_model.score(tie_break_policy="abstain", output_str=False)
report.update({"accuracy": {"precision": None, "recall": None, "f1-score": report["accuracy"], "support": report['macro avg']['support']}})
df = pd.DataFrame(report).transpose()
df = df.astype(float).applymap(lambda x: '{:,.2f}'.format(x))
df['support'] = df['support'].astype(float).apply(lambda x: '{:,g}'.format(x))
df = df.replace(['nan', 'None'], '')
print(df.to_string())

                precision recall f1-score support
efficacy             0.47   0.47     0.47    0.47
fscore_cautious      0.34   0.34     0.34    0.34
coverage             0.22   0.22     0.22    0.22
negative             0.90   0.50     0.64      94
positive             0.66   0.95     0.78      97
accuracy                             0.73     191
macro avg            0.78   0.72     0.71     191
weighted avg         0.78   0.73     0.71     191


In [7]:
print(flyingsquid_model.score(tie_break_policy="abstain", output_str=True))

              precision    recall  f1-score   support

    negative       0.90      0.50      0.64        94
    positive       0.66      0.95      0.78        97

    coverage                           0.22       191
    fscore_cautious                           0.34       191
    efficacy                           0.47       191
    accuracy                           0.73       191
   macro avg       0.78      0.72      0.71       191
weighted avg       0.78      0.73      0.71       191



In [ ]:
accuracy_row_pattern = re.compile(r'(\s+)(\w*)(\s+)(0\.[0-9][0-9])(.*)')

def get_metrics_row(row, metric, value):
    f_string = f'(\\1){metric}(\\3){value}(\\5)'
    print(f_string)
    res = accuracy_row_pattern.search(row)
    first_half = [res.group(1), res.group(2)]
    spaces = res.group(3)
    second_half = [res.group(4), res.group(5)]
    first_half[1] = metric
    second_half[0] = value
    first_half = " ".join(first_half)
    second_half = " ".join(second_half)
    diff = len(first_half + spaces + second_half) - len(first_half + second_half)
    if diff:
        output = first_half + " " * diff + second_half
    else:
        output = first_half + " " + second_half
    return output

get_metrics_row(lines[idx], "efficacy", "0.87")

In [ ]:
lines[idx]

In [ ]:
# get your training records with the predictions of the label model
records_for_training = flyingsquid_model.predict()

# log the records to a new dataset in Rubrix
rb.log(records_for_training, name="flyingsquid_results")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.inputs["text"], "label": flyingsquid_model.weak_labels.label2int[rec.prediction[0][0]]}
        for rec in records_for_training
    ]
)

In [ ]:
epoxy.score()

In [ ]:
is_max, is_tie, prediction, annotation = flyingsquid_model._get_score_objects(verbose=True)

coverage = len(prediction[~is_tie]) / len(prediction)

coverage

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.label.values
)

In [ ]:
# compute the test accuracy
accuracy = classifier.score(
    X=test_df.text.tolist(),
    y=test_df.label.tolist()
)

print(f"Test accuracy: {accuracy}")

# Epoxy

In [ ]:
from sentence_transformers import SentenceTransformer

class SentenceTransformerModel(object):
    
    def __init__(self, embedding_model_name):
        self.embedding_model = SentenceTransformer(embedding_model_name)
    
    def __call__(self, records):
        texts = [ x.inputs["text"] for x in records ]
        embeddings = self.embedding_model.encode(texts)
        return embeddings

In [ ]:
from rubrix.labeling.text_classification import Epoxy
embedding_model_name = "average_word_embeddings_glove.840B.300d"
model = SentenceTransformerModel(embedding_model_name)

embeddings = model(weak_labels.records())

epoxy = Epoxy(weak_labels, embeddings)
epoxy.fit()

In [ ]:
epoxy._thresholds

In [ ]:
# get your training records with the predictions of the label model
records_for_training = epoxy.predict()

# log the records to a new dataset in Rubrix
rb.log(records_for_training, name="epoxy_results_2")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.inputs["text"], "label": epoxy.weak_labels.label2int[rec.prediction[0][0]]}
        for rec in records_for_training
    ]
)

In [ ]:
# preview training data
training_data

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.label.values
)

In [ ]:
# compute the test accuracy
accuracy = classifier.score(
    X=test_df.text.tolist(),
    y=test_df.label.tolist()
)

print(f"Test accuracy: {accuracy}")